# Creating the model that will be used for classifying financial sentences on Streamlit API

## Downloading required packages

In [5]:

import requests
import pandas as pd
import re
import nltk
import numpy as np
import torch
import transformers as ppb
from numpy import linspace
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import seaborn as sns
import pysentiment2 as ps
from keras.models import model_from_json
from keras.layers import Dense
from keras.models import Sequential
#from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
#from keras.optimizers import SGD doesnt work
import scikeras
from scikeras.wrappers import KerasClassifier
import keras.optimizers
import os
import tensorflow as tf
#from keras.optimizers import RMSprop doesnt work
#from prettytable import PrettyTable
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from tensorflow.keras.optimizers import RMSprop
import pickle
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm  # better for Jupyter

In [6]:
os.chdir('C:/Users/xeb15154/OneDrive - University of Strathclyde/PhD/Excel Files (Data)')
df = pd.read_csv('new_classified_audio_test1(verified).csv')
del df['Unnamed: 0.1']
del df['Unnamed: 0']
df.shape

(18425, 42)

In [8]:
df_classified = df[df['sentiment'].notnull()]
df_classified.shape

(10000, 42)

In [10]:
df_classified['Sentiment_manual'].value_counts()

Sentiment_manual
 0.0    1469
 1.0     304
-1.0     111
Name: count, dtype: int64

In [11]:
#downloading the sampled df
os.chdir('H:/')
sampled_df = pd.read_csv('sampled_df_equalweighted.csv')

In [12]:
sampled_df

,Unnamed: 0,new,speech,sentiment,Sentiment_manual,name,session,time,id,quarter,...,Shimmer local,Shimmer local dB,Shimmer apq3,Shimmer apq5,Shimmer apq11,Shimmer dda,Mean autocorrelation,Mean NHR,Mean HNR,Audio Length
0,0,JPM_57713_000645,And if you think about 11%plus tier one capita...,1.0,NaN,Marianne Lake,question_answer,11/10/2013 16:36,JPM_57713,3,...,12.130,1.138,4.807,7.172,12.167,14.422,0.862177,0.194742,10.197,17.513991
1,1,CVX_181025_000460,So there is no primary need on balance sheet,0.0,0.0,Pierre Breber,question_answer,29/01/2021 23:14,CVX_181025,4,...,14.836,1.244,6.570,9.223,13.112,19.710,0.786955,0.311092,6.546,7.341958
2,2,LMT_171182_000325,If theyre not sure theyre going to get paid fo...,-1.0,NaN,James Taiclet,question_answer,21/07/2020 21:21,LMT_171182,2,...,12.822,1.228,5.390,7.708,13.904,16.169,0.834549,0.244605,8.601,40.501746
3,3,AXP_22181_000347,You may have seen that we were rated 1 in cust...,1.0,NaN,Daniel T. Henry,question_answer,23/10/2009 04:51,AXP_22181,3,...,14.428,1.260,6.723,9.102,12.561,20.169,0.840758,0.206720,7.759,24.655147
4,4,UNP_149565_000810,That’s all from me,0.0,NaN,Cherilyn Radbourne,question_answer,18/04/2019 21:18,UNP_149565,1,...,17.217,1.483,7.715,10.271,18.200,23.144,0.751303,0.386504,5.624,5.481270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2101,HON_86730_000093,What was the process?,0.0,0.0,Scott Davis,question_answer,28/07/2015 19:02,HON_86730,2,...,21.467,1.755,10.289,16.527,11.007,30.866,0.788477,0.300832,6.411,58.961875
2102,2102,BLK_77575_000300,But I do believe in this divergent world and t...,1.0,NaN,Larry Fink,question_answer,16/01/2015 04:09,BLK_77575,4,...,21.663,1.711,10.139,18.930,27.440,30.416,0.741760,0.402328,5.312,2.973515
2103,2103,LLY_28408_000376,I wan to remind the investment community once ...,-1.0,-1.0,Derica Rice,question_answer,21/10/2010 20:35,LLY_28408,3,...,19.008,1.620,9.075,12.276,18.522,27.224,0.770155,0.347879,6.351,8.772698
2104,2104,CVX_37942_000410,That was the quarter recognized 2 of those dr...,-1.0,NaN,Patricia E. Yarrington,question_answer,27/04/2012 19:51,CVX_37942,1,...,12.630,1.035,5.229,6.777,8.244,15.688,0.871881,0.174371,10.346,36.462494


In [36]:
sampled_df['sentiment'].value_counts()

sentiment
 1.0    702
 0.0    702
-1.0    702
Name: count, dtype: int64

In [37]:
sampled_df['speech']

0       And if you think about 11%plus tier one capita...
1            So there is no primary need on balance sheet
2       If theyre not sure theyre going to get paid fo...
3       You may have seen that we were rated 1 in cust...
4                                      That’s all from me
                              ...                        
2101                                What was the process?
2102    But I do believe in this divergent world and t...
2103    I wan to remind the investment community once ...
2104    That was  the quarter recognized 2 of those dr...
2105    On cap rate compression, we are already seeing...
Name: speech, Length: 2106, dtype: object

In [39]:
#splitting the data into train and test sets using a stratified split to ensure the above proportions are kept
#for both the training and test sets

X = sampled_df['speech']
y = sampled_df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                stratify=y, 
                                                test_size=0.20,
                                                random_state=9)

df_train = pd.DataFrame(X_train)
df_test = pd.DataFrame(X_test)

In [44]:
y_test

1060    1.0
144     0.0
1639   -1.0
643     1.0
369     1.0
       ... 
718    -1.0
757     1.0
1777    1.0
97      0.0
797     1.0
Name: sentiment, Length: 422, dtype: float64

## Encoding sentences using BERT 

In [45]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModel.from_pretrained("yiyanghkust/finbert-tone")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_cls_embeddings(texts, batch_size=4):
    model.eval()
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]

        encoded = tokenizer.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding="longest",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        input_ids = encoded["input_ids"].to(device)
        attention_mask = encoded["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] token
            all_embeddings.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

In [46]:
#generate embeddings for Xtrain and Xtest
X_train_embeddings = get_cls_embeddings(X_train.tolist(), batch_size=4)
X_test_embeddings = get_cls_embeddings(X_test.tolist(), batch_size=4)

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [00:37<00:00,  2.82it/s]


In [51]:
#structing the data to apply to the model
def making_label(st):
    if st == 0:
        return 0
    elif st == 1:
        return 1
    else:
        return 2
#0 = neutral, 1 = positive, 2 = negative

#0 = neutral, 1 = positive, 2 = negative
labels_train = y_train.apply(making_label)
labels_test = y_test.apply(making_label)

In [54]:
y_train = tf.keras.utils.to_categorical(labels_train)

In [55]:
y_train

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [56]:
n_cols = X_train_embeddings.shape[1]
input_shape = (n_cols,)

In [57]:
model_text = Sequential()
model_text.add(Dense(768, activation = 'relu', input_shape = input_shape))
model_text.add(Dense(3, activation = 'softmax', kernel_initializer = 'he_uniform'))
model_text.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_text.fit(X_train_embeddings, y_train, epochs = 10, batch_size =100)
# loss: 0.7134 - accuracy: 0.7994

C:\Users\xeb15154\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7019 - loss: 0.8282
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7791 - loss: 0.5507
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8254 - loss: 0.4494
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8521 - loss: 0.3966
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8866 - loss: 0.3293
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8931 - loss: 0.2885
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9287 - loss: 0.2311
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9471 - loss: 0.1930
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9650 - loss: 0.1571
Epoch 10/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9709 - loss: 0.1301


In [58]:
#evaluate the model
predictions = model_text.predict(X_test_embeddings)
predictions = pd.DataFrame(predictions)
y_pred = predictions.idxmax(axis=1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [66]:
testing_df = pd.DataFrame(y_test)
testing_df['sentiment'] = testing_df['sentiment'].astype(int)
testing_df['sentiment'] = testing_df['sentiment'].apply(making_label)
testing_df['sentiment_predictions'] = list(y_pred)
testing_df

,sentiment,sentiment_predictions
1060,1,1
144,0,0
1639,2,2
643,1,1
369,1,1
...,...,...
718,2,2
757,1,1
1777,1,1
97,0,0


In [67]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(testing_df['sentiment'], testing_df['sentiment_predictions'])
print(f'Accuracy: {accuracy:.4f}')  # prints accuracy as a decimal, e.g., 0.85


Accuracy: 0.7133


In [76]:
#Changing working directory
os.chdir('C:/Users/xeb15154/OneDrive - University of Strathclyde/Financial Sentiment Analysis Project/')

#Saving the model

with open("model_text.pkl", "wb") as f:
    pickle.dump(model_text, f)